In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
# 워킹데이 리스트 불러오기
date_df = pd.read_csv('../../../Stock_Data/Data/stock_df/date_list.csv')
del date_df['Unnamed: 0']
del date_df['article']
date_df = date_df[date_df['Date'] <= '2019-12-31']

## 휴일 날짜 변경, 날짜, 카테고리별 Summary, 카테고리 원핫인코딩

#### 1) for문 사용해서 전체 종목 돌릴때

In [45]:
kospi20=['SK텔레콤','LG전자','이마트','기아차','SK하이닉스','한국전력',\
'현대백화점','셀트리온','KB금융','LG화학','아모레퍼시픽','기업은행','현대모비스','대우조선해양','GS건설',\
'S-Oil','CJ제일제당', '삼성전자','현대차','대한항공']

for index, kospi in enumerate(kospi20):
    #print(index)
    num = index+1
    article_df = pd.read_csv('C:/py/stock_trading_agent/article_df/selected20/{}_df.csv'.format(kospi))
    del article_df['Unnamed: 0']
    article_df = article_df.drop_duplicates('Title',keep = 'first')
    article_df = article_df.drop(article_df[article_df['Date'] == '2019.12.31'].index)
    article_df = article_df.dropna(axis=0)
    article_df = article_df.reset_index(drop=True)
    #print(len(article_df))
    for i in range(len(article_df)):
        #print(i)
        article_df['Date'][i] = article_df['Date'][i].replace('.','-')
    article_df = article_df.sort_values('Date')

    
    while True:
        index = article_df['Date'].isin(date_df['Date']) == False
        holidays = article_df['Date'][index]
        #print(len(holidays))
        if len(holidays) == 0:
            break
        else:
            for i in range(len(holidays)):
                holiday = holidays.iloc[i]
                conv_hd = datetime.datetime.strptime(holiday,'%Y-%m-%d')
                not_holiday = str(conv_hd + datetime.timedelta(days=1))[:10]
                holidays.iloc[i] = not_holiday
            article_df['Date'][index] = holidays 
    
    merge_df = pd.merge(date_df, article_df, left_on=['Date','Category'], right_on=['Date','Category'], how='left')
    
    # one-hot encoding
    merge_df['politics']=0
    merge_df['economics']=0
    merge_df['IT']=0
    merge_df['finance']=0
    merge_df['realEstate']=0
    merge_df['international']=0
    merge_df['social']=0
    merge_df['life']=0
    merge_df['golf']=0
    merge_df['ent']=0

    for i in range(len(merge_df)):
        if merge_df['Category'][i] == 2:
            merge_df['politics'][i] = 1
        elif merge_df['Category'][i] == 3:
            merge_df['economics'][i] = 1
        elif merge_df['Category'][i] == 4:
            merge_df['IT'][i] = 1
        elif merge_df['Category'][i] == 5:
            merge_df['finance'][i] = 1
        elif merge_df['Category'][i] == 6:
            merge_df['realEstate'][i] = 1
        elif merge_df['Category'][i] == 7:
            merge_df['international'][i] = 1
        elif merge_df['Category'][i] == 8:
            merge_df['social'][i] = 1
        elif merge_df['Category'][i] == 9:
            merge_df['life'][i] = 1
        elif merge_df['Category'][i] == 10:
            merge_df['golf'][i] = 1
        elif merge_df['Category'][i] == 11:
            merge_df['ent'][i] = 1
        #print(i)

    del merge_df['Category']    
    merge_df.to_csv('C:/py/stock_trading_agent/article_df/selected20/{index}. {name}_df_V2.csv'.format(index=num,name=kospi), index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/py/stock_trading_agent/article_df/selected20/SK텔레콤_df.csv'

#### 2) 한 종목만 변경

In [123]:
article_df = pd.read_csv('../../../Sentiment Analysis/handmade_sentitest/trainingset/CJ제일제당_traintestdata_3300.csv')
del article_df['Unnamed: 0']
article_df = article_df.drop_duplicates('Title',keep = 'first')
article_df = article_df.drop(article_df[article_df['Date'] == '2019.12.31'].index)
article_df = article_df.dropna(axis=0)
article_df = article_df.reset_index(drop=True)

# Date 형식 변경 및 sorting
for i in range(len(article_df)):
    article_df['Date'][i] = article_df['Date'][i].replace('.','-')
article_df = article_df.sort_values('Date')

# date_df에 없는 날짜 그 다음 날짜로 변경
while True:
    index = article_df['Date'].isin(date_df['Date']) == False
    holidays = article_df['Date'][index]
    #print(len(holidays))
    if len(holidays) == 0:
        break
    else:
        for i in range(len(holidays)):
            holiday = holidays.iloc[i]
            conv_hd = datetime.datetime.strptime(holiday,'%Y-%m-%d')
            not_holiday = str(conv_hd + datetime.timedelta(days=1))[:10]
            holidays.iloc[i] = not_holiday
        article_df['Date'][index] = holidays 

# cj기사에 이상치 변경
article_df.loc[article_df['label']=='-'] = -1

# 긍정일때 3, 부정일때 1, 중립일때 2로 변경 
for i in range(len(article_df)):
    if pd.isnull(article_df['label'][i]) == False:
        article_df['label'][i] = int(article_df['label'][i]) + 2      
        
# article_df
article_df2 = article_df[['Date','Category','label']].groupby(by=['Date', 'Category'], as_index=False).sum()
        
# article_df와 date_df merge
merge_df = pd.merge(date_df, article_df2, left_on=['Date','Category'], right_on=['Date','Category'], how='left')

# one-hot encoding
merge_df['politics']=0
merge_df['economics']=0
merge_df['IT']=0
merge_df['finance']=0
merge_df['realEstate']=0
merge_df['international']=0
merge_df['social']=0
merge_df['life']=0
merge_df['golf']=0
merge_df['ent']=0

for i in range(len(merge_df)):
    if merge_df['Category'][i] == 2:
        merge_df['politics'][i] = 1
    elif merge_df['Category'][i] == 3:
        merge_df['economics'][i] = 1
    elif merge_df['Category'][i] == 4:
        merge_df['IT'][i] = 1
    elif merge_df['Category'][i] == 5:
        merge_df['finance'][i] = 1
    elif merge_df['Category'][i] == 6:
        merge_df['realEstate'][i] = 1
    elif merge_df['Category'][i] == 7:
        merge_df['international'][i] = 1
    elif merge_df['Category'][i] == 8:
        merge_df['social'][i] = 1
    elif merge_df['Category'][i] == 9:
        merge_df['life'][i] = 1
    elif merge_df['Category'][i] == 10:
        merge_df['golf'][i] = 1
    elif merge_df['Category'][i] == 11:
        merge_df['ent'][i] = 1
    #print(i)

del merge_df['Category']

#파일명 변경해서 돌리기!
merge_df.to_csv('cj_final.csv', index=False)

/home/test/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/test/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/test/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/test/.local/lib/python3.6/site-

### 컬럼수 줄이기: 기사수가 많은 5개의 카테고리만 유지

economics, IT, finance, social, politics만 유지

In [124]:
summary = pd.read_csv('cj_final.csv')

In [150]:
temp = summary[pd.isnull(summary['label'])==False]
temp[temp.columns[2:12]].sum().sort_values(ascending=False).head(5)

economics    624
IT           555
finance      336
social        57
politics      18
dtype: int64

### 5개 카테고리에 해당하는 행, 열 추출해서 csv파일 만들기

In [172]:
is_economics = summary['economics']==1
is_IT = summary['IT']==1
is_finance = summary['finance']==1
is_social = summary['social']==1
is_politics = summary['politics']==1

senti_df = summary[is_economics|is_IT|is_finance|is_social|is_politics]\
                [['Date', 'label', 'economics', 'IT', 'finance', 'social', 'politics']]

senti_df.to_csv('../../../Sentiment Analysis/Data/senti_CJ제일제당.csv', index=False)